# Notebook to Append Financial Data and Returns

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import statsmodels.formula.api as smf
import quandl
import requests
import json
quandl.ApiConfig.api_key = 'M51b-yW2sUripq1s85N3'
intrinio_key = 'OmM2NzM3NTUyMGU3NGY0ODU1MDRmM2M4YTUzMmE5Nzcx'

## Read in Data

In [2]:
df = pd.read_excel("output_cleaned_watson.xlsx")

#convert string dates to datetimes
for i in range(0, len(df)):
    df.iloc[i, 5] = datetime.strptime(str(df.iloc[i,5]), '%Y-%m-%d %H:%M:%S')

## Summary of Data

## Total Company and CEO Stats

In [3]:
print("Number of Transcripts: %d" %len(df))
print("Number of Companies: %d" %len(df['Ticker'].unique()))
print("Average Transcripts per CEO: %.2f" % (len(df) / len(df['Ticker'].unique())))

Number of Transcripts: 1160
Number of Companies: 63
Average Transcripts per CEO: 18.41


## Stock Returns

In [4]:
df['Next_Date'] = np.nan
df['Time_Delta'] = np.nan
for i in range(0, len(df)):
    #find the next date for that company
    temp_df = df[df['Ticker'] == df.iloc[i, 1]]
    min_delta = timedelta(days=100000)
    next_date = ''
    min_found = False
    for j in range(0, len(temp_df)):
        delta = temp_df.iloc[j, 5] - df.iloc[i, 5]
        if delta.days > 0:
            if delta.days < min_delta.days:
                min_delta = delta
                next_date = temp_df.iloc[j, 5]
                min_found = True
    if min_found:
        df.loc[i, 'Next_Date'] = next_date
        df.loc[i, 'Time_Delta'] = min_delta.days
    

with_stock_df = df.dropna(axis = 0).copy()
with_stock_df.head()

,CEO,Ticker,Sector,Quarter,Year,Date,CEOScore,AnalystScore,OtherExecScore,CEO_Word_Count,...,Extraversion_Raw,Agreeableness_Raw,Neuroticism_Raw,Openness_Percentile,Concientiousness_Percentile,Extraversion_Percentile,Agreeableness_Percentile,Neuroticism_Percentile,Next_Date,Time_Delta
0,TomÂ Greco,AAP,Consumer Discretionary,Q3,2016,2016-11-15 22:57:00,0.151,0.128,0.130,6672,...,0.561661,0.706720,0.521500,0.954784,0.790463,0.594356,0.186081,0.831548,2017-02-21 12:41:00,97.0
1,ThomasÂ R.Â Greco,AAP,Consumer Discretionary,Q4,2016,2017-02-21 12:41:00,0.158,0.176,0.160,6383,...,0.572874,0.697931,0.533246,0.952018,0.756394,0.687953,0.132744,0.878734,2017-05-24 08:00:00,91.0
2,TomÂ Greco,AAP,Consumer Discretionary,Q1,2017,2017-05-24 08:00:00,0.127,0.095,0.152,4824,...,0.587895,0.695040,0.533590,0.957695,0.855521,0.795789,0.117763,0.879958,2017-08-15 13:20:00,83.0
3,TomÂ Greco,AAP,Consumer Discretionary,Q2,2017,2017-08-15 13:20:00,0.145,0.087,0.119,2523,...,0.581642,0.697082,0.547846,0.934888,0.912710,0.753815,0.128213,0.923305,2017-11-14 14:45:00,91.0
4,ThomasÂ R.Â Greco,AAP,Consumer Discretionary,Q3,2017,2017-11-14 14:45:00,0.138,0.153,0.154,5464,...,0.559625,0.701363,0.524434,0.933496,0.827466,0.576576,0.152166,0.844314,2018-02-21 17:14:00,99.0


In [5]:
def get_returns(price_data):
    try:
        t0_close = price_data.iloc[0,3]
        t1_open = price_data.iloc[1, 0]
        t20_close = price_data.iloc[1, 3]
        t25_close = price_data.iloc[5, 3]
        t30_close = price_data.iloc[30, 3]
        t35_close = price_data.iloc[45, 3]
        days_in_sample = len(price_data) - 2
        ret_20d = (t20_close - t1_open)/t1_open
        ret_25d = (t25_close - t1_open)/t1_open
        ret_30d = (t30_close - t1_open)/t1_open
        ret_35d = (t35_close - t1_open)/t1_open
        return [ret_20d, ret_25d, ret_30d, ret_35d]
    except:
        print(price_data)

In [6]:
for i in with_stock_df.index:
    if i % 50 == 0:
        print(i)
    #this should be long enough to get 45 trading days of price data
    next_date = with_stock_df.loc[i, 'Date'] + timedelta(days=90)
    px_df = quandl.get("EOD/" + with_stock_df.loc[i, 'Ticker'], start_date=with_stock_df.loc[i, 'Date'], end_date=next_date)
   # print(with_stock_df.loc[i, 'Ticker'])
    get_returns(px_df)
    try:
        ret_list = get_returns(px_df)
        with_stock_df.loc[i, 'twenty_ret'] = ret_list[0]
        with_stock_df.loc[i, 'twentyfive_ret'] = ret_list[1]
        with_stock_df.loc[i, 'thirty_ret'] = ret_list[2]
        with_stock_df.loc[i, 'thirtyfive_ret'] = ret_list[3]
    except:
        print(with_stock_df.loc[i, 'Ticker'])
with_stock_df.dropna(axis=0, inplace=True)
with_stock_df.head()

0
50
100
150
200
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150


,CEO,Ticker,Sector,Quarter,Year,Date,CEOScore,AnalystScore,OtherExecScore,CEO_Word_Count,...,Concientiousness_Percentile,Extraversion_Percentile,Agreeableness_Percentile,Neuroticism_Percentile,Next_Date,Time_Delta,twenty_ret,twentyfive_ret,thirty_ret,thirtyfive_ret
0,TomÂ Greco,AAP,Consumer Discretionary,Q3,2016,2016-11-15 22:57:00,0.151,0.128,0.130,6672,...,0.790463,0.594356,0.186081,0.831548,2017-02-21 12:41:00,97.0,-0.001409,0.043380,0.043318,0.028001
1,ThomasÂ R.Â Greco,AAP,Consumer Discretionary,Q4,2016,2017-02-21 12:41:00,0.158,0.176,0.160,6383,...,0.756394,0.687953,0.132744,0.878734,2017-05-24 08:00:00,91.0,-0.006595,-0.025694,-0.110116,-0.093070
2,TomÂ Greco,AAP,Consumer Discretionary,Q1,2017,2017-05-24 08:00:00,0.127,0.095,0.152,4824,...,0.855521,0.795789,0.117763,0.879958,2017-08-15 13:20:00,83.0,-0.007744,0.030827,-0.233609,-0.171353
3,TomÂ Greco,AAP,Consumer Discretionary,Q2,2017,2017-08-15 13:20:00,0.145,0.087,0.119,2523,...,0.912710,0.753815,0.128213,0.923305,2017-11-14 14:45:00,91.0,0.057186,0.106747,0.147528,0.014441
4,ThomasÂ R.Â Greco,AAP,Consumer Discretionary,Q3,2017,2017-11-14 14:45:00,0.138,0.153,0.154,5464,...,0.827466,0.576576,0.152166,0.844314,2018-02-21 17:14:00,99.0,-0.025019,-0.053231,0.061535,0.256574


## Add Financial Data

In [7]:
def parse_JSON(json_results, df, i):
    json_text = json.loads(response.text)
    data = json_text['data']
    for tag in data:
        item = tag['tag']
        value = tag['value']
        df.loc[i, item] = value
    return df

In [8]:
for i in with_stock_df.index:
    if i % 5 == 0:
        print(i)
    try:
        ticker = with_stock_df.loc[i, 'Ticker']
        date = with_stock_df.loc[i, 'Date'].strftime('%Y-%m-%d')
        base_apicall = 'https://api.intrinio.com/financials/standardized?'
        statement = 'calculations'
        fin_type = 'TTM'
        query = base_apicall + 'identifier=' + ticker + '&statement=' + statement + '&type=' + fin_type + '&date=' + date  +\
            '&api_key=' + intrinio_key
        response = requests.get(query)
        with_stock_df = parse_JSON(response.text, with_stock_df, i)
    except:
        print(response.text)

0
5
10
15
25
30
35
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
140
145
150
160
165
170
175
180
185
195
200
205
210
215
220
225
230
235
240
245
255
260
265
270
275
280
285
295
300
305
310
315
325
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155


## Add Fed Funds Rate

In [9]:
#get EFFR from Quandl from minimum date minus 120 days to latest date
max_date = datetime(1900, 1, 1)
min_date = datetime(2020, 1, 1)
for i in with_stock_df.index:
    if with_stock_df.loc[i, 'Date'] < min_date:
        min_date = with_stock_df.loc[i, "Date"]
    if with_stock_df.loc[i, 'Date'] > max_date:
        max_date = with_stock_df.loc[i, "Date"]
min_date = min_date - timedelta(days=130)
ff_df = quandl.get("FRED/EFFR" , start_date = min_date, end_date = max_date)
t10_df = quandl.get("FRED/DGS10" , start_date = min_date, end_date = max_date)
indices = []
for i in ff_df.index:
    indices.append(i.date().strftime("%Y-%m-%d"))
ff_df.set_index(pd.Index(indices), inplace=True)
indices = []
for i in t10_df.index:
    indices.append(i.date().strftime("%Y-%m-%d"))
t10_df.set_index(pd.Index(indices), inplace=True)

for i in with_stock_df.index:
    date = with_stock_df.loc[i, 'Date']
    date_30 = (with_stock_df.loc[i, 'Date'] - timedelta(days=30))
    date_60 = (with_stock_df.loc[i, 'Date'] - timedelta(days=60))
    date_90 = (with_stock_df.loc[i, 'Date'] - timedelta(days=90))
    date_120 = (with_stock_df.loc[i, 'Date'] - timedelta(days=120))
    try:
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value
    except:
        while date.strftime("%Y-%m-%d") not in ff_df.index:
            date = date - timedelta(days=1)
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'FFR_30'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value - ff_df.loc[date_30.strftime("%Y-%m-%d")].Value
    except:
        while date_30.strftime("%Y-%m-%d") not in ff_df.index:
            date_30 = date_30 - timedelta(days=1)
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value- ff_df.loc[date_30.strftime("%Y-%m-%d")].Value

    try:
        with_stock_df.loc[i, 'FFR_60'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value - ff_df.loc[date_60.strftime("%Y-%m-%d")].Value
    except:
        while date_60.strftime("%Y-%m-%d") not in ff_df.index:
            date_60 = date_60 - timedelta(days=1)
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value- ff_df.loc[date_60.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'FFR_90'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value - ff_df.loc[date_90.strftime("%Y-%m-%d")].Value
    except:
        while date_90.strftime("%Y-%m-%d") not in ff_df.index:
            date_90 = date_90 - timedelta(days=1)
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value- ff_df.loc[date_90.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'FFR_120'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value - ff_df.loc[date_120.strftime("%Y-%m-%d")].Value
    except:
        while date_120.strftime("%Y-%m-%d") not in ff_df.index:
            date_120 = date_120 - timedelta(days=1)
        with_stock_df.loc[i, 'FFR_0'] = ff_df.loc[date.strftime("%Y-%m-%d")].Value- ff_df.loc[date_120.strftime("%Y-%m-%d")].Value

In [10]:
for i in with_stock_df.index:
    date = with_stock_df.loc[i, 'Date']
    date_30 = (with_stock_df.loc[i, 'Date'] - timedelta(days=30))
    date_60 = (with_stock_df.loc[i, 'Date'] - timedelta(days=60))
    date_90 = (with_stock_df.loc[i, 'Date'] - timedelta(days=90))
    date_120 = (with_stock_df.loc[i, 'Date'] - timedelta(days=120))
    try:
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value
    except:
        while date.strftime("%Y-%m-%d") not in t10_df.index:
            date = date - timedelta(days=1)
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'T10_30'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value - t10_df.loc[date_30.strftime("%Y-%m-%d")].Value
    except:
        while date_30.strftime("%Y-%m-%d") not in t10_df.index:
            date_30 = date_30 - timedelta(days=1)
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value- t10_df.loc[date_30.strftime("%Y-%m-%d")].Value

    try:
        with_stock_df.loc[i, 'T10_60'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value - t10_df.loc[date_60.strftime("%Y-%m-%d")].Value
    except:
        while date_60.strftime("%Y-%m-%d") not in t10_df.index:
            date_60 = date_60 - timedelta(days=1)
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value- t10_df.loc[date_60.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'T10_90'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value - t10_df.loc[date_90.strftime("%Y-%m-%d")].Value
    except:
        while date_90.strftime("%Y-%m-%d") not in t10_df.index:
            date_90 = date_90 - timedelta(days=1)
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value- t10_df.loc[date_90.strftime("%Y-%m-%d")].Value
    try:
        with_stock_df.loc[i, 'T10_120'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value - t10_df.loc[date_120.strftime("%Y-%m-%d")].Value
    except:
        while date_120.strftime("%Y-%m-%d") not in t10_df.index:
            date_120 = date_120 - timedelta(days=1)
        with_stock_df.loc[i, 'T10_0'] = t10_df.loc[date.strftime("%Y-%m-%d")].Value- t10_df.loc[date_120.strftime("%Y-%m-%d")].Value

## Binary Factor for Trump Tax Law

In [12]:
tax_date = datetime(2017, 12, 22)
for i in with_stock_df.index:
    if with_stock_df.loc[i, 'Date'] < tax_date:
        with_stock_df.loc[i, 'Tax_Law'] = 0
    else:
        with_stock_df.loc[i, 'Tax_Law'] = 1
with_stock_df['Tax_Law']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       1.0
7       0.0
8       0.0
9       0.0
10      0.0
11      1.0
12      1.0
14      0.0
15      0.0
16      0.0
17      0.0
18      1.0
19      1.0
21      0.0
22      0.0
23      0.0
24      0.0
25      1.0
26      1.0
28      0.0
29      0.0
30      0.0
31      0.0
32      0.0
34      0.0
       ... 
1130    0.0
1131    0.0
1132    0.0
1133    0.0
1134    0.0
1135    0.0
1136    0.0
1137    0.0
1138    0.0
1139    0.0
1140    0.0
1141    0.0
1142    0.0
1143    0.0
1144    0.0
1145    0.0
1146    0.0
1147    0.0
1148    0.0
1149    0.0
1150    0.0
1151    0.0
1152    0.0
1153    0.0
1154    0.0
1155    0.0
1156    0.0
1157    0.0
1158    0.0
1159    0.0
Name: Tax_Law, Length: 1097, dtype: float64

## Write back to CSV

In [ ]:
with_stock_df.to_csv('Final_Data.csv')

In [ ]:
issues_arr = ['']
for column in with_stock_df.columns:
    for i in with_stock_df.index:
        if with_stock_df.loc[i, column] == 'nm':
            issues_arr.append([with_stock_df.loc[i, 'Ticker'], with_stock_df.loc[i, 'Date'], column])
issues_arr.pop(0)
df = pd.DataFrame(data=issues_arr, columns=['Ticker', 'Date', 'Item'])
df.to_csv('problems.csv')

In [ ]:
with_stock_df.head()

In [15]:
features_list = ['CEOScore_sqrt', 'AnalystScore_pwr', 'OtherExecScore', 'CEO_Word_Count_sqrt', 'Analyst_Word_Count', 'OtherExec_Word_Count_pwr', 
                 'CEO_Sent_Count_sqrt', 'Analyst_Sent_Count', 'OtherExec_Sent_Count_sqrt', 'CEO_Pos_Fin_sqrt', 'CEO_Neg_Fin', 'CEO_Fin_Score2', 
                 'Openness_Raw', 'Concientiousness_Raw', 'Extraversion_Raw', 'Agreeableness_Raw', 'Neuroticism_Raw',
                 'marketcap', 'pricetobook', 'dividendyield', 'revenuegrowth', 'freecashflow', 'ebitda', 'Tax_Law',
                  'FFR_120', 'FFR_90', 'FFR_60', 'FFR_30', 'FFR_0']

In [16]:
with_stock_df['FFR_120']

0       0.01
1       0.25
2       0.25
3       0.25
4       0.00
5       0.26
7       0.26
8       0.25
9       0.25
10      0.00
11      0.26
12      0.28
14      0.00
15      0.26
16      0.25
17      0.10
18      0.26
19      0.28
21      0.26
22      0.25
23      0.25
24      0.00
25      0.26
26      0.27
28      0.26
29      0.50
30       NaN
31      0.25
32      0.25
34       NaN
        ... 
1130   -0.06
1131   -0.02
1132   -0.02
1133    0.02
1134    0.08
1135   -0.02
1136     NaN
1137   -0.03
1138    0.05
1139   -0.06
1140   -0.02
1141   -0.01
1142     NaN
1143    0.02
1144    0.04
1145    0.09
1146   -0.02
1147     NaN
1148     NaN
1149   -0.02
1150   -0.07
1151     NaN
1152    0.09
1153    0.02
1154   -0.01
1155   -0.03
1156   -0.01
1157   -0.07
1158   -0.01
1159   -0.02
Name: FFR_120, Length: 1097, dtype: float64